In [3]:
!pip install python-telegram-bot --upgrade
!pip install dextools-python

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [4]:
api_key = "b5ZO15hnCV9dnJr7XOQZIaGMWXCb6y5J7bXSoJvS"

In [5]:
from dextools_python import DextoolsAPIV2
import pandas as pd
import time
import datetime

dextools = DextoolsAPIV2(api_key, plan="trial")

In [6]:
# Get Tokens

In [7]:
def get_liquidity_pools_sorted_by_date_df(chain='solana', start_date=None, end_date=None, num_pages=3):
    all_tokens = []
    for page in range(0, num_pages):
        response = dextools.get_pools(chain, from_=end_date, to=start_date, page=page, order="desc")
        all_tokens.extend(response["data"]['results'])
        time.sleep(2)
    
    # Flatten the nested 'socialInfo' and convert data to DataFrame
    tokens_df = pd.json_normalize(all_tokens)
    
    return tokens_df

In [8]:
def run_get_tokens():
    tokens = get_liquidity_pools_sorted_by_date_df()
    return tokens

In [9]:
import requests

def get_rug_check_data(token_address):
    # Base API URL
    base_url = 'https://api.rugcheck.xyz/v1/tokens/'
    
    # Complete API URL with the token address
    api_url = f'{base_url}{token_address}/report'
    # Send GET request
    response = requests.get(api_url)
    # Check if the request was successful
    if response.status_code == 200:
        # Parse the JSON response into a pandas DataFrame
        data = response.json()
        df_top_holders = pd.json_normalize(data)
    else:
        good_resp = False
        # Send GET request
        while not good_resp:
            time.sleep(2)
            response = requests.get(api_url)
            # Check if the request was successful
            if response.status_code == 200:
                # Parse the JSON response into a pandas DataFrame
                data = response.json()
                df_top_holders = pd.json_normalize(data)
                good_resp = True
    return df_top_holders

In [10]:
def add_top_10_holders_percentage(df):
    # Assuming df['topHolders'] is a string representation of the topHolders list
    # Convert the string to a list of dictionaries if needed
    if isinstance(df['topHolders'].iloc[0], str):
        df['topHolders'] = df['topHolders'].apply(eval)

    # Normalize the nested JSON data for topHolders and calculate the sum of the 'pct' of the top 10 holders
    top_holders_df = pd.json_normalize(df['topHolders'].iloc[0])
    top_10_holders_pct = top_holders_df.nlargest(10, 'pct')['pct'].sum()

    # Add the top 10 holders percentage to the DataFrame
    df['top_10_holders_percentage'] = top_10_holders_pct

    # Drop the original topHolders column if you want to remove it
    #df.drop(columns=['topHolders'], inplace=True)

    return df

In [11]:
import pandas as pd
import ast
import json

def calculate_liquidity(df):
    # Convert string representation of lists back into actual lists for the 'topHolders' column
    df['topHolders'] = df['topHolders'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
    
    # Initialize total percentage and liquidity
    total_pct = 0
    liquidity_usd = 0

    # Iterate over each row for topHolders to sum percentages and calculate liquidity
    for index, row in df.iterrows():
        # Calculate total percentage from topHolders
        top_holders = row['topHolders']
        total_pct += sum(holder['pct'] for holder in top_holders)
        
        # Calculate liquidity from markets (assuming 'markets' is a stringified JSON in the DataFrame)
        markets_data = json.loads(row['markets']) if isinstance(row['markets'], str) else row['markets']
        for market in markets_data:
            liquidity_usd += market['lp']['baseUSD'] + market['lp']['quoteUSD']

    
    
    # Add the calculated values as new columns to the DataFrame
    df['total_percentage_of_top_holders'] = total_pct
    df['total_liquidity_usd'] = liquidity_usd
    return df

In [12]:
def add_each_top_10_holders_percentage(df):
    # Assuming the 'topHolders' column is a list of dictionaries with the 'pct' key
    for index, row in df.iterrows():
        # Sort the top holders by percentage in descending order
        sorted_holders = sorted(row['topHolders'], key=lambda x: x['pct'], reverse=True)
        # Get the top ten holders
        top_ten_holders = sorted_holders[:10]
        for i, holder in enumerate(top_ten_holders):
            # Create a new column for each top holder's percentage
            df.at[index, f'top_holder_{i+1}_pct'] = holder['pct']

    return df

In [13]:
import pandas as pd
import ast

def calculate_market_cap(df):
    # Initialize an empty list to store market cap results
    market_caps = []
    
    # Loop through each row in the DataFrame
    for index, row in df.iterrows():
        # Check if markets is a string and convert it to a list if necessary
        if isinstance(row['markets'], str):
            markets = ast.literal_eval(row['markets'])
        else:
            markets = row['markets']
        
        # Initialize total market cap for the row
        total_market_cap = 0

        # Loop through each market to calculate its market cap
        for market in markets:
            base_price = market['lp']['basePrice']  # Extract the base price for the market
            # Calculate market cap using total supply and base price
            market_cap = row['token.supply'] / (10 ** row['token.decimals']) * base_price  
            total_market_cap += market_cap  # Sum up the market cap for the row

        # Append the calculated market cap for the row to the list
        market_caps.append(total_market_cap)
    
    # Add the market cap list as a new column to the DataFrame
    df['market_cap'] = market_caps
    
    # Return the updated DataFrame
    return df

In [14]:
def add_social_info(df):
    # Create new columns with empty strings for each social info key
    social_keys = ['discord', 'facebook', 'github', 'instagram', 'linkedin', 
                   'medium', 'reddit', 'telegram', 'tiktok', 'twitter', 
                   'website', 'youtube', 'bitbucket']
    for key in social_keys:
        df[key] = None

    # Iterate over the DataFrame and update with social info
    for index, row in df.iterrows():
        token_data = dextools.get_token('solana', row['mint'])
        social_info = token_data['data']['socialInfo']
        for key in social_keys:
            if key in social_info:
                df.at[index, key] = social_info[key]
    
    return df

In [15]:
def calculate_lp_locked_pct_from_df(df):
    # Initialize a list to store the locked percentage values for each row
    lp_locked_pcts = []
    
    # Loop through each row in the DataFrame
    for index, row in df.iterrows():

        if isinstance(row['markets'], str):
            markets = ast.literal_eval(row['markets'])
        else:
            markets = row['markets']
        # Convert the string representation of the list back to a list
        try:
            # Initialize a temporary variable to store the LP Locked Percentage
            lp_locked_pct = 0
            # Loop through the markets to find the 'lpLockedPct' value
            for market in markets:
                if 'lp' in market and 'lpLockedPct' in market['lp']:
                    lp_locked_pct = market['lp']['lpLockedPct']
                    break
            # Append the percentage to the list
            lp_locked_pcts.append(lp_locked_pct)
        except ValueError:
            # If there is an issue with converting the string to a list, append a default value
            lp_locked_pcts.append(None)
    
    # Add the list as a new column to the DataFrame
    df['lpLockedPct'] = lp_locked_pcts
    return df

In [16]:
def calculate_mc_prices(df, base):
    # Create new columns for the USD and base currency prices
    df['price_usd'] = 0.0
    df['price_base'] = 0.0
    
    # Iterate over the DataFrame
    for index, row in df.iterrows():
        if isinstance(row['markets'], str):
            markets = ast.literal_eval(row['markets'])
        else:
            markets = row['markets']
        
        # If there are markets available, get the first one
        if markets:
            market = markets[0]
            lp_info = market.get('lp', {})
            
            # Extract the base price in the base currency
            base_price = lp_info.get('basePrice', 0)
            # Extract the base price in USD
            base_usd = lp_info.get('baseUSD', 0)
            
            # Update the DataFrame with the new information
            df.at[index, 'price_usd'] = base_usd
            df.at[index, f'price_base_{base}'] = base_price
    
    return df

In [17]:
def is_a_sniper_liquidity_pool(liquidity_info_df, top_10_holders_pct_thrshld, total_liquidity_usd_thrshld, top_holder_1_pct_thrshld, lpLockedPct_thrshld):
    # if top 10 holders hold greater than the thrshld
    liquidity_info_df = liquidity_info_df.loc[liquidity_info_df['top_10_holders_percentage'] >= top_10_holders_pct_thrshld]
    # if total liquidity in usd is less than the thrshld
    liquidity_info_df = liquidity_info_df.loc[liquidity_info_df['total_liquidity_usd'] <= total_liquidity_usd_thrshld]
    # if top holder holds greater than the thrshld
    liquidity_info_df = liquidity_info_df.loc[liquidity_info_df['top_holder_1_pct'] >= top_holder_1_pct_thrshld]
    # if the liquidity locked percentage is less than the thrshld
    liquidity_info_df = liquidity_info_df.loc[liquidity_info_df['lpLockedPct'] <= lpLockedPct_thrshld]
    # if does not have a value in the social media columns
    social_keys = ['discord', 'facebook', 'github', 'instagram', 'linkedin', 
                   'medium', 'reddit', 'telegram', 'tiktok', 'twitter', 'youtube', 'bitbucket']
    liquidity_info_df = liquidity_info_df.dropna(subset=social_keys)
    liquidity_info_df = liquidity_info_df.dropna(subset=["website"])
    return liquidity_info_df

In [20]:
import math
# run the scraper on the entire set of addresses
def get_liquidity_pools_signals(chain, base, num_lp, time_offset=0, start_date=None, end_date=None):
    # calculate how many pages to grab from the API by getting the chain info
    # and using the pageSize with num_lp to figure out how many pages to grab from the following API call
    if not start_date:
        start_date = datetime.datetime.now()
    if not end_date:
        end_date = start_date - datetime.timedelta(minutes=15)
    
    start_date_str = start_date.strftime('%Y-%m-%dT%H:%M:%S')
    end_date_str = end_date.strftime('%Y-%m-%dT%H:%M:%S')
    
    response = response = dextools.get_pools(chain, from_=end_date_str, to=start_date_str, order="desc")
    page_size = response["data"]["pageSize"]
    total_pages = response["data"]["totalPages"]
    # calculate number of pages to pass into the next function
    num_pages_to_scrape = math.ceil(num_lp / page_size)
    if num_pages_to_scrape > total_pages:
        print("Error: Too many liquidity pools requested for the timeframe!")
        return False
    print(f'Total number of pages: {total_pages}\nTotal number of LPs per page: {page_size}')
    print(f'Total number of LPs to scrape: {num_lp}\nTotal number of pages to scrape: {num_pages_to_scrape}')
    # Update message
    print("Grabbing the liquidity pools 🤑")
    # set a timeout to satisify num request per sec
    time.sleep(2)
    # Get recently launched Liquidiy Pools and extract the token addresses
    lp_df = get_liquidity_pools_sorted_by_date_df(chain, start_date=start_date, end_date=end_date, num_pages=num_pages_to_scrape)
    mc_addr_lst = lp_df["mainToken.address"].tolist()
    # update message
    print("COMPLETED 😎")
    print("Starting Rug Checker for each address 🔬\n\n")
    # rest before the rug checking
    time.sleep(2)
    # filter the list based off the startegy
    filter_mc_addr_lst = []
    # for each address in mc_addr_lst get the rugchecker info, social media info, and run the analysis.
    # if coin passes the analysis, pass the address to filter_mc_addr_lst
    for coin_addr in mc_addr_lst[:num_lp]:
        print(f'Running Rug Checker for coin with addr == {coin_addr} 🤞🏾')
        # get rugchecker info
        rug_check_df = get_rug_check_data(coin_addr)
        # Add the top 10 holders percentage of coins
        rug_check_df = add_top_10_holders_percentage(rug_check_df)
        # Add each of the 10 top holders percentages to the df
        rug_check_df = add_each_top_10_holders_percentage(rug_check_df)
        # Calculate and add the liquidity in USD
        rug_check_df = calculate_liquidity(rug_check_df)
        # Calaculate and add the Liquidity Locked Percentage
        rug_check_df = calculate_lp_locked_pct_from_df(rug_check_df)
        # Calculate and add the market capitalization
        rug_check_df = calculate_market_cap(rug_check_df)
        # Calculate the price of the meme coin in USD and the base coin
        rug_check_df = calculate_mc_prices(rug_check_df, base)
        # Add the social info
        rug_check_df = add_social_info(rug_check_df)
        # check if we can snipe this LP
        rug_check_df = is_a_sniper_liquidity_pool(rug_check_df, 50, 1000, 5, 70)
        if rug_check_df.shape[0] == 0:
            print(f'👎🏾👎🏾👎🏾 DROPPED coin with addr == {coin_addr} 👎🏾👎🏾👎🏾\n')
        else:
            print(f'🟢🟢🟢 SNIPE SIGNAL FOR COIN WITH ADDR == {coin_addr} 🟢🟢🟢\n')
            filter_mc_addr_lst.append({
                "coin_addr": coin_addr,
                "rug_check_info": rug_check_df.head(1).to_json(orient="records")
            })
        # rest after each iteration
        time.sleep(2)
    print(filter_mc_addr_lst)
    return True
        

In [ ]:
get_liquidity_pools_signals("solana", "SOL", 10)

Total number of pages: 2
Total number of LPs per page: 20
Total number of LPs to scrape: 10
Total number of pages to scrape: 1
Grabbing the liquidity pools 🤑
COMPLETED 😎
Starting Rug Checker for each address 🔬


Running Rug Checker for coin with addr == 7MUKpncTCa7BbHEF2ouSzJFtyBFpX8JooVzAbhXiymCh 🤞🏾
👎🏾👎🏾👎🏾 DROPPED coin with addr == 7MUKpncTCa7BbHEF2ouSzJFtyBFpX8JooVzAbhXiymCh 👎🏾👎🏾👎🏾

Running Rug Checker for coin with addr == 44nziFXpnw5fEsML4g3pzpkHSKid5BiirZwevBP8XXth 🤞🏾
🟢🟢🟢 SNIPE SIGNAL FOR COIN WITH ADDR == 44nziFXpnw5fEsML4g3pzpkHSKid5BiirZwevBP8XXth 🟢🟢🟢

Running Rug Checker for coin with addr == GfJHKwLnHprhPBpnuVxaN7Jt6C4ZYZEeZMs94dDEzqyb 🤞🏾
👎🏾👎🏾👎🏾 DROPPED coin with addr == GfJHKwLnHprhPBpnuVxaN7Jt6C4ZYZEeZMs94dDEzqyb 👎🏾👎🏾👎🏾

Running Rug Checker for coin with addr == DyugVhjgSgSTbGqmmMrgwPQF5uA8y8VX2wCo42hJRGVE 🤞🏾
👎🏾👎🏾👎🏾 DROPPED coin with addr == DyugVhjgSgSTbGqmmMrgwPQF5uA8y8VX2wCo42hJRGVE 👎🏾👎🏾👎🏾

Running Rug Checker for coin with addr == Ei3WAHK3EkJPZjfScv9g8XGp51CdQh7V8EmbKw6